In [1]:
import numpy as np

merged_bar = np.load("data/merged_bar.npy")
merged_bar.shape


(8074, 6)

In [2]:
import pandas as pd

from custom_indicators.all_features import feature_bundle

features = feature_bundle(merged_bar, sequential=True)
df_features = pd.DataFrame(features, index=merged_bar[:, 0].astype(int))
print(df_features.shape)
print(f"max nan: {df_features.isna().sum().max()}")
df_features.head()

(8074, 3447)
max nan: 68


,adx_7,adx_7_dt,adx_7_ddt,adx_7_lag1,adx_7_lag2,adx_7_lag3,adx_7_lag4,adx_7_lag5,adx_7_lag6,adx_7_lag7,...,williams_r_ddt_lag10,williams_r_ddt_lag11,williams_r_ddt_lag12,williams_r_ddt_lag13,williams_r_ddt_lag14,williams_r_ddt_lag15,williams_r_ddt_lag16,williams_r_ddt_lag17,williams_r_ddt_lag18,williams_r_ddt_lag19
1577940300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1577953680000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1577973960000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1577987940000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1578007260000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df_features.iloc[68:]
df.isna().sum().max()

0

In [5]:
df.to_parquet("data/features.parquet")